In [2]:
import pandas as pd
pd.set_option('display.max_columns',None)

import sys
sys.path.append("../../")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [6]:
def get_edu(new_school, new_academy, new_daycare, new_kinder):
    # 학교
    school = pd.read_csv('../../' + new_school, encoding='cp949')
    tmp = school.loc[school['소재지지번주소'].str.contains('평택시')]
    high = tmp.loc[tmp['학교급구분'] == '고등학교'][['학교명','소재지지번주소','위도','경도']]
    high.rename(columns={'소재지지번주소':'주소', '학교명':'이름'},inplace=True)

    mid = tmp.loc[tmp['학교급구분'] == '중학교'][['학교명','소재지지번주소','위도','경도']]
    mid.rename(columns={'소재지지번주소':'주소', '학교명':'이름'},inplace=True)

    ele = tmp.loc[tmp['학교급구분'] == '초등학교'][['학교명','소재지지번주소','위도','경도']]
    ele.rename(columns={'소재지지번주소':'주소', '학교명':'이름'},inplace=True)


    # 학원
    academy = pd.read_csv('../../' + new_academy, encoding='cp949', low_memory=False)
    tmp2 = academy.loc[academy['시설도로명주소'].str.contains('평택시')]
    aca = tmp2[['학원명','시설도로명주소']]
    aca = aca.reset_index().drop(columns=['index'])

    aca['위도'] = pd.NA
    aca['경도'] = pd.NA

    for i, address in enumerate(aca['시설도로명주소']):
        lng, lat = get_coordinates(address)
        aca.at[i, '위도'] = lat
        aca.at[i, '경도'] = lng

    aca.rename(columns={'시설도로명주소':'주소', '학원명':'이름'},inplace=True)


    # 어린이집
    daycare = pd.read_csv('../../' + new_daycare, encoding='cp949')

    care = daycare.loc[daycare['소재지도로명주소'].str.contains('평택시')][['어린이집명','소재지도로명주소']].reset_index().drop(columns=['index'])
    care['위도'] = pd.NA
    care['경도'] = pd.NA

    for i, address in enumerate(care['소재지도로명주소']):
        lng, lat = get_coordinates(address)
        care.at[i, '위도'] = lat
        care.at[i, '경도'] = lng

    care.rename(columns={'소재지도로명주소':'주소', '어린이집명':'이름'},inplace=True)


    # 유치원
    kinder = pd.read_csv('../../' + new_kinder, encoding='cp949')

    kin = kinder.loc[kinder['정제지번주소'].str.contains('평택시')][['유치원명','정제지번주소','정제WGS84위도','정제WGS84경도']].dropna()

    kin.rename(columns={'정제지번주소':'주소','정제WGS84위도':'위도','정제WGS84경도':'경도','유치원명':'이름'},inplace=True)


    return high, mid, ele, aca, kin, care

In [7]:
new_school = 'new_data/교육시설/청주대학교 지방교육재정연구원_초중등학교위치_20240322.csv'
new_academy = 'new_data/교육시설/학원교습소정보.csv'
new_daycare = 'new_data/교육시설/어린이집현황(제공표준).csv'
new_kinder = 'new_data/교육시설/유치원현황.csv'

high, mid, ele, aca, kin, care = get_edu(new_school, new_academy, new_daycare, new_kinder)

In [8]:
high.to_csv('../../preprocessed/교육시설/고등학교.csv', index=False)
mid.to_csv('../../preprocessed/교육시설/중학교.csv', index=False)
ele.to_csv('../../preprocessed/교육시설/초등학교.csv', index=False)
aca.to_csv('../../preprocessed/교육시설/학원.csv', index=False)
kin.to_csv('../../preprocessed/교육시설/유치원.csv', index=False)
care.to_csv('../../preprocessed/교육시설/어린이집.csv', index=False)